In [7]:
#Translation of Alphabets to Braille
import LettersToBrailleData

In [61]:
CAPITAL = chr(10272) #  .
NUMBER = chr(10300) #  ⠼
UNRECOGNIGED = '?'
#There is  braille symbol for only open(") and close(") quotation not for a generic quote(")
#therefore we must keep track of what the last quotation was so that we may convert the generic quotation to a specific one
open_quotes = True 

In [19]:
#Split up a sentence based on whitespace(" ") and new line("\n") chars.
def extract_words(string):
    #WO = string.split(".")
    words = string.split(" ")
    result = []
    for word in words:
        temp = word.split("\n")
        for item in temp:
            result.append(item)
    return result

In [123]:
#Return True if a char is Braille.
def is_braille(char):
    if len(char) > 1:
        return False
    return char in LettersToBrailleData.letters or char in LettersToBrailleData.numbers or char in LettersToBrailleData.punctuation or char in LettersToBrailleData.contractions or char == CAPITAL or char == NUMBER

In [32]:
#Remove punctuation a word. as durgesh." >durgesh
def trim(word):
    while len(word) is not 0 and not word[0].isalnum():
        word = word[1:]
    while len(word) is not 0 and not word[-1].isalnum():
        word = word[:-1]
    return word

In [132]:
#Replace each group of numbers in a world to their respective braille representation.
def numbers_handler(word):
    if word == "":
        return word
    result = word[0]
    if word[0].isdigit():
        result = NUMBER + LettersToBrailleData.numbers.get(word[0])
    for i in range(1, len(word)):
        if word[i].isdigit() and word[i-1].isdigit():
            result += LettersToBrailleData.numbers.get(word[i])
        elif word[i].isdigit():
            result += NUMBER + LettersToBrailleData.numbers.get(word[i])
        else:
            result +=word[i]
    return result

In [134]:
numbers_handler("Dur 5 ")

'Dur ⠼⠑ '

In [51]:
#putting the escape code before each capital letter.
def capital_letter_handler(word):
    if word == "":
        return word
    result = ""
    for char in word:
        if char.isupper():
            result += CAPITAL + char.lower()
        else:
            result += char.lower()
    return result

In [161]:
capital_letter_handler("Durgesh")

'⠠durgesh'

In [135]:
capital_letter_handler("Dur 5 G")

'⠠dur 5 ⠠g'

In [55]:
#Find the UTF code of a particular character. Used what an unidentified char is found.
def find_utf_code(char):
    if len(char)!=1:
        return -1
    for i in range(0,55000):
        if char == chr(i):
            return i

In [155]:
#Convert an alphabetic char to braille.
def char_to_braille(char):
    if is_braille(char):
        return char
    elif char == "\n":
        return "\n"
    elif char == "\"":
        global open_quotes
        if open_quotes:
            open_quotes = not open_quotes
            return LettersToBrailleData.punctuation.get("“")
        else:
            open_quotes = not open_quotes
            return LettersToBrailleData.punctuation.get("”")
    elif char in LettersToBrailleData.letters and char.isupper():
        return CAPITAL + LettersToBrailleData.letters.get(char)
    elif char in LettersToBrailleData.letters:
        return LettersToBrailleData.letters.get(char)
    elif char in LettersToBrailleData.punctuation:
        return LettersToBrailleData.puntuation.get(char)
    else:
        print("Unrecognised sysmbol:", char,"With UTF code:", find_utf_code(char))
        return UNRECOGNIZED

In [156]:
#Convert an alphabetic word to braille
def word_to_braille(word):
    if word in LettersToBrailleData.contractions:
        return LettersToBrailleData.contractions.get(word)
    else:
        result = ""
        for char in word:
            result += char_to_braille(char)
        return result

In [157]:
#Translate a trimmed word to braille then re-attach the shaving.
def build_braille_word(trimmed_word, shavings, index, braille):
    if shavings == "":
        braille += word_to_braille(trimmed_word)
    else:
        for i in range(0, len(shavings)):
            if i == index and trimmed_word is not "":
                braille += word_to_braille(trimmed_word)
            braille += word_to_braille(shavings[i])
        if index == len(shavings): #If the shavings are all at the begining.
            braille += word_to_braille(trimmed_word)
    return braille

In [158]:
def translate(string):
    braille = ""
    words = extract_words(string)
    for word in words:
        word = numbers_handler(word)
        word = capital_letter_handler(word)
        trimmed_word = trim(word) # Remove puntuation
        untrimmed_word = word
        index = untrimmed_word.find(trimmed_word)
        shavings = untrimmed_word.replace(trimmed_word, "")
        braille = build_braille_word(trimmed_word, shavings, index, braille)+ " "
    return braille[:-1] #Remove Final Space that was added

In [166]:
char = "Durgesh"